In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-02 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-02 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-02 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-02 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-02 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [11]:
size = 5

## Random Projections

In [12]:
(
    swmhau_network_grp_kfold_5,
    best_swmhau_network_grp_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_5_kfold_best_model.csv


In [13]:
swmhau_network_grp_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1815314/572217191.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.639591   
                                                                                      0.0005         0.633844   
                                                                                      0.0010         0.636143   
                                                                         0.2          0.0001         0.638378   
                                                                                      0.0005         0.634866   
                                                                                      0.0010         0.628416   
                                                          (512, 512)     0.1          0.0001         0.624968   
                                                                                      0.0005         0.631928   
                                                                                      0.0010         0.617816   
                                                                         0.2          0.0001         0.627905   
                                                                                      0.0005         0.642273   
                                                                                      0.0010         0.630396   
           12              3         10        1          (256, 256)     0.1          0.0001         0.639017   
                                                                                      0.0005         0.613857   
                                                                                      0.0010         0.629374   
                                                                         0.2          0.0001         0.624968   
                                                                                      0.0005         0.631928   
                                                                                      0.0010         0.622925   
                                                          (512, 512)     0.1          0.0001         0.623435   
                                                                                      0.0005         0.633333   
                                                                                      0.0010         0.629438   
                                                                         0.2          0.0001         0.632439   
                                                                                      0.0005         0.634355   
                                                                                      0.0010         0.636909   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.487949   
                                                                                      0.0005         0.516875   
                                                                                      0.0010         0.519909   
                                                                         0.2          0.0001         0.502046   
                                                                                      0.0005         0.515103   
                                                                                      0.0010         0.508531   
                                                          (512, 512)     0.1          0.0001         0.495681   
                                                                                      0.0005         0.513005   
                                

In [14]:
best_swmhau_network_grp_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.630077,0.516690,"[0.7516192197620124, 0.448870164687859, 0.3495...",0.509405,"[0.7843445457403332, 0.41887062187276625, 0.325]",0.527732,"[0.7215153267784846, 0.4834983498349835, 0.378...",None,0.660726,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.634100,0.518451,"[0.7567890182035213, 0.447443762781186, 0.3511...",0.510301,"[0.781750924784217, 0.44363341443633414, 0.305...",0.532473,"[0.7333718912666282, 0.4513201320132013, 0.412...",None,0.661850,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.637356,0.515483,"[0.762341117351463, 0.44048112816258816, 0.343...",0.508947,"[0.7796251511487303, 0.44286905754795663, 0.30...",0.526157,"[0.7458068247541931, 0.4381188118811881, 0.394...",None,0.665382,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


in path (with speaker): 0.5210764081829135 (2:22)

in path (no speaker): 0.5207288898117722 (2:30)

in input (no speaker): 0.5142427437088648 (2:40)

both (no speaker): 0.5129526518731312 (3:40)

(w/ time_encoding) in path (with speaker): 0.520978678702496

(w/ time_encoding) in path (no speaker): 0.5201665510110508

(w/ time_encoding) in input (no speaker): 0.5190188331551432

(w/ time_encoding) both (no speaker): 0.5091037146573926

In [15]:
# with speaker in path
best_swmhau_network_grp_kfold_5["f1"].mean()

0.5168746613941936

In [16]:
# no speaker in path
best_swmhau_network_grp_kfold_5["f1"].mean()

0.5168746613941936

In [17]:
# no speaker in input
best_swmhau_network_grp_kfold_5["f1"].mean()

0.5168746613941936

In [18]:
# no speaker both
best_swmhau_network_grp_kfold_5["f1"].mean()

0.5168746613941936

In [19]:
best_swmhau_network_grp_kfold_5["precision"].mean()

0.5095510790222296

In [20]:
best_swmhau_network_grp_kfold_5["recall"].mean()

0.5287873202203582

In [21]:
np.stack(best_swmhau_network_grp_kfold_5["f1_scores"]).mean(axis=0)

array([0.75691645, 0.44559835, 0.34810918])

In [22]:
np.stack(best_swmhau_network_grp_kfold_5["precision_scores"]).mean(axis=0)

array([0.78190687, 0.43512436, 0.311622  ])

In [23]:
np.stack(best_swmhau_network_grp_kfold_5["recall_scores"]).mean(axis=0)

array([0.73356468, 0.45764576, 0.39515152])

# w=11

In [24]:
size = 11

## Random Projections

In [25]:
(
    swmhau_network_grp_kfold_11,
    best_swmhau_network_grp_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_11_kfold_best_model.csv


In [26]:
swmhau_network_grp_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1815314/2867464325.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.623755   
                                                                                      0.0005         0.630715   
                                                                                      0.0010         0.626054   
                                                                         0.2          0.0001         0.633333   
                                                                                      0.0005         0.644444   
                                                                                      0.0010         0.631545   
                                                          (512, 512)     0.1          0.0001         0.619732   
                                                                                      0.0005         0.630843   
                                                                                      0.0010         0.629183   
                                                                         0.2          0.0001         0.635377   
                                                                                      0.0005         0.636590   
                                                                                      0.0010         0.615006   
           12              3         10        1          (256, 256)     0.1          0.0001         0.616475   
                                                                                      0.0005         0.631290   
                                                                                      0.0010         0.616731   
                                                                         0.2          0.0001         0.624457   
                                                                                      0.0005         0.639017   
                                                                                      0.0010         0.626820   
                                                          (512, 512)     0.1          0.0001         0.617625   
                                                                                      0.0005         0.643423   
                                                                                      0.0010         0.618519   
                                                                         0.2          0.0001         0.621201   
                                                                                      0.0005         0.627905   
                                                                                      0.0010         0.625990   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.475777   
                                                                                      0.0005         0.514733   
                                                                                      0.0010         0.508195   
                                                                         0.2          0.0001         0.474308   
                                                                                      0.0005         0.520699   
                                                                                      0.0010         0.505735   
                                                          (512, 512)     0.1          0.0001         0.472774   
                                                                                      0.0005         0.509631   
                                

In [27]:
best_swmhau_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.636398,0.519323,"[0.7597566404511056, 0.4386252045826514, 0.359...",0.511658,"[0.7802499238037184, 0.43506493506493504, 0.31...",0.531155,"[0.7403123192596877, 0.44224422442244227, 0.41...",None,0.665543,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.642337,0.519569,"[0.764809384164223, 0.4426778242677824, 0.3512...",0.514147,"[0.7757287328970851, 0.44906621392190155, 0.31...",0.527796,"[0.7541931752458069, 0.43646864686468645, 0.39...",None,0.671162,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.654598,0.523206,"[0.7764303038949921, 0.42485676509475545, 0.36...",0.523981,"[0.7662630245001408, 0.45600756859035, 0.34967...",0.524551,"[0.786871023713129, 0.3976897689768977, 0.3890...",None,0.679512,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [28]:
best_swmhau_network_grp_kfold_11["f1"].mean()

0.5206991573326142

In [29]:
best_swmhau_network_grp_kfold_11["precision"].mean()

0.516595688633151

In [30]:
best_swmhau_network_grp_kfold_11["recall"].mean()

0.5278340479122137

In [31]:
np.stack(best_swmhau_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.76699878, 0.4353866 , 0.3597121 ])

In [32]:
np.stack(best_swmhau_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.77408056, 0.44671291, 0.3289936 ])

In [33]:
np.stack(best_swmhau_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.76045884, 0.42546755, 0.39757576])

# w=20

In [34]:
size = 20

## Random Projections

In [35]:
(
    swmhau_network_grp_kfold_20,
    best_swmhau_network_grp_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_20_kfold_best_model.csv


In [36]:
swmhau_network_grp_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1815314/1955217834.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.624713   
                                                                                      0.0005         0.636079   
                                                                                      0.0010         0.636335   
                                                                         0.2          0.0001         0.622925   
                                                                                      0.0005         0.644125   
                                                                                      0.0010         0.629566   
                                                          (512, 512)     0.1          0.0001         0.623691   
                                                                                      0.0005         0.622414   
                                                                                      0.0010         0.628672   
                                                                         0.2          0.0001         0.624074   
                                                                                      0.0005         0.638506   
                                                                                      0.0010         0.630907   
           12              3         10        1          (256, 256)     0.1          0.0001         0.614304   
                                                                                      0.0005         0.630843   
                                                                                      0.0010         0.628289   
                                                                         0.2          0.0001         0.602107   
                                                                                      0.0005         0.630332   
                                                                                      0.0010         0.632184   
                                                          (512, 512)     0.1          0.0001         0.608301   
                                                                                      0.0005         0.634036   
                                                                                      0.0010         0.619413   
                                                                         0.2          0.0001         0.621264   
                                                                                      0.0005         0.626309   
                                                                                      0.0010         0.628608   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.477561   
                                                                                      0.0005         0.512811   
                                                                                      0.0010         0.513733   
                                                                         0.2          0.0001         0.490455   
                                                                                      0.0005         0.516541   
                                                                                      0.0010         0.512475   
                                                          (512, 512)     0.1          0.0001         0.480730   
                                                                                      0.0005         0.510398   
                                

In [37]:
best_swmhau_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.648467,0.520089,"[0.7708004023566606, 0.4235294117647058, 0.365...",0.518672,"[0.7660668380462725, 0.4487534626038781, 0.341...",0.523709,"[0.7755928282244072, 0.400990099009901, 0.3945...",None,0.671805,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.637739,0.512580,"[0.760932944606414, 0.4277360066833751, 0.3490...",0.508611,"[0.7671957671957672, 0.43316412859560066, 0.32...",0.517859,"[0.7547715442452284, 0.42244224422442245, 0.37...",None,0.676622,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.646169,0.516955,"[0.766906474820144, 0.43080829454083786, 0.353...",0.516761,"[0.7631729667812142, 0.4422241529105126, 0.344...",0.517487,"[0.7706766917293233, 0.41996699669966997, 0.36...",None,0.677103,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [38]:
best_swmhau_network_grp_kfold_20["f1"].mean()

0.5165413246788632

In [39]:
best_swmhau_network_grp_kfold_20["precision"].mean()

0.5146812590170516

In [40]:
best_swmhau_network_grp_kfold_20["recall"].mean()

0.519685297428914

In [41]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.76621327, 0.4273579 , 0.3560528 ])

In [42]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.76547852, 0.44138058, 0.33718467])

In [43]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.76701369, 0.41446645, 0.37757576])

# w=35

In [44]:
size = 35

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_35,
    best_swmhau_network_grp_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


In [ ]:
swmhau_network_grp_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_35

In [ ]:
best_swmhau_network_grp_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_35["recall_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_80,
    best_swmhau_network_grp_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_80

In [ ]:
best_swmhau_network_grp_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["recall_scores"]).mean(axis=0)

## w=110

In [ ]:
size = 110

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_110,
    best_swmhau_network_grp_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_20

In [ ]:
best_swmhau_network_grp_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)